In [7]:
import json
import re
import urllib.request
import datetime
from typing import Dict, List
from bs4 import BeautifulSoup
from ollama import chat, ChatResponse
from IPython.display import display, Markdown

# For version checking of installed packages:
import bs4
import ollama
import IPython

# Built-in modules
print("json: built-in")
print("re: built-in")
print("urllib.request: built-in")
print("datetime: built-in")
print("typing: built-in")

# Third-party libraries with version attributes
print("beautifulsoup4 version:", bs4.__version__)

try:
    print("ollama version:", ollama.__version__)
except AttributeError:
    print("ollama version not available (or not set)")

print("IPython version:", IPython.__version__)


json: built-in
re: built-in
urllib.request: built-in
datetime: built-in
typing: built-in
beautifulsoup4 version: 4.12.3
ollama version not available (or not set)
IPython version: 8.25.0


In [ ]:
from ollama import chat, ChatResponse
import json
import re
from IPython.display import display, Markdown

# --- Prompt Baselines ---
INITIAL_RESPONSE_PROMPT = (
    "You are an expert on the topic: {research_topic}. Provide an extensive, detailed, and comprehensive answer "
    "to the research question. In your answer, highlight any areas or gaps that might require further exploration."
)

FIND_GAP_PROMPT = (
    "You are a group of 3 experts on the topic: {research_topic}."
    "Think step by step on the following assay: "
    "<ASSAY>"
    "{assay}"
    "</ASSAY>"
    "Provide one new topic to explore to fill a knowledge gap in the assay."
    "Based on the gaps identified in your answer, generate a JSON object with the following keys:\n"
    '   - "query": "The search query string."\n'
    '   - "aspect": "The aspect of the topic being addressed by this query."\n'
    '   - "rationale": "Why this query will help fill the gap."\n'
    "Provide only the JSON structure."
)

FINALIZE_RESPONSE_PROMPT = (
    "You are a team of expert on the topic: {research_topic}. Your goal is to analyze the text provided in the <TEXT></TEXT> tags "
    "and create an extensive, detailed, and comprehensive report using the information provided. Aim to 500 words per section."
    "Your thesis is formatted in markdown and have:\n"
    "1. Title\n"
    "2. Introduction\n"
    "3. Discussion\n"
    "4. Gaps / Further research\n"
    "<TEXT>"
    "{notes}"
    "</TEXT>"
)

# --- Helper: Remove <THINK> Tags ---
def remove_think_tags(text: str) -> str:
    """
    Remove any text enclosed in <THINK>...</THINK> tags.
    The regex is case-insensitive.
    """
    return re.sub(r"<\s*THINK\s*>.*?<\s*/\s*THINK\s*>", "", text, flags=re.DOTALL | re.IGNORECASE).strip()

# --- Configuration & State Management ---
class Configuration:
    def __init__(self, ollama_base_url: str, local_llm: str, fetch_full_page: bool,
                 max_research_loops: int, max_fetch_pages: int, max_token_per_search: int):
        self.ollama_base_url = ollama_base_url
        self.local_llm = local_llm
        self.fetch_full_page = fetch_full_page
        self.max_research_loops = max_research_loops
        self.max_fetch_pages = max_fetch_pages
        self.max_token_per_search = max_token_per_search

def query_local_llm(state: dict, config: Configuration, prompt="") -> str:
    """
    Generate an extensive answer for the research topic.
    The answer should also indicate potential gaps for further research.
    """
    message = {"role": "user", "content": prompt}
    response: ChatResponse = chat(model=config.local_llm, messages=[message])
    initial_response = remove_think_tags(response.message.content.strip())
    state["initial_response"] = initial_response
    state["assay"] = initial_response
    return initial_response

def initialize_state(research_topic: str) -> dict:
    """
    Initialize the research state with the given topic.
    """
    return {
        "research_topic": research_topic,
        "initial_response": "",       # The original extensive answer.
        "assay":"",
        "search_query": research_topic
    }

def extract_json_from_llm_output(text: str):
    """
    Attempt to extract a JSON object from the provided text.
    This function supports two formats:
      1. JSON enclosed in triple backticks with the tag "json" (e.g., ```json { ... } ```).
      2. JSON enclosed in triple backticks without the tag (e.g., ``` { ... } ```).
      3. A plain JSON string.
    
    Returns:
        Parsed JSON object.
        
    Raises:
        ValueError if no valid JSON structure can be found or parsed.
    """
    # Define patterns for JSON enclosed in triple backticks.
    patterns = [
        r"```json\s*(\{.*?\})\s*```",  # with "json" tag (case-insensitive)
        r"```(\{.*?\})```"             # without the tag
    ]
    
    matches = []
    for pattern in patterns:
        found = re.findall(pattern, text, flags=re.DOTALL | re.IGNORECASE)
        if found:
            matches.extend(found)
    
    # If no triple-backtick JSON is found, check if the whole text is JSON.
    if not matches:
        stripped = text.strip()
        if stripped.startswith("{") and stripped.endswith("}"):
            matches.append(stripped)
    
    if not matches:
        print("<ERROR>:\n%s\n</ERROR>" % text)
        raise ValueError("No JSON structure found in the provided text.")
    
    # Try parsing each candidate.
    for match in matches:
        try:
            json_data = json.loads(match)
            return json_data
        except json.JSONDecodeError:
            continue
    
    print("<ERROR>:\n%s\n</ERROR>" % text)
    raise ValueError("Found JSON-like structure, but could not parse it.")


def main():
    config = Configuration(
        ollama_base_url="http://localhost:11434",  # Your Ollama URL
        local_llm="qwq",                      # Default LLM is "llama3.2"
        fetch_full_page=True,                      # Fetch full page content if needed
        max_research_loops=3,                      # Number of research iterations
        max_fetch_pages=5,                         # Number of pages to fetch per search
        max_token_per_search=4000                  # Token limit per search processing
    )

    # Step 1: Get the research question from the user
    research_topic = input("Enter your research question: ")
    print("#### Research Question ####\n")
    print(research_topic)
    print("\n")
    state = initialize_state(research_topic)

    # Step 2: Generate an initial explanation using the local LLM. This answer will be the first instance of the assay
    prompt_initial = INITIAL_RESPONSE_PROMPT.format(research_topic=state["research_topic"])
    initial_explanation = query_local_llm(state, config, prompt_initial)
    print("#### Initial Explanation ####\n")
    print(initial_explanation)
    print("\n")

    print("#### Thinking Process ####\n")
    for i in range(config.max_research_loops):
        print("  >> Find gap... \n")
        # Step 3: Evaluate the current assay and generate a follow-up question
        prompt_gap = FIND_GAP_PROMPT.format(research_topic=state["research_topic"], assay=state["assay"])
        followup_question_llm = query_local_llm(state, config, prompt_gap)
        followup_question_json = extract_json_from_llm_output(followup_question_llm)
        print(f"  >> Iteration {i+1} - Follow-up Question: {followup_question_json["query"]}\n")

        print("  >> Generate new response based on follow-up question... \n")
        # Step 4: Reiterate on a new topic
        prompt_follow = INITIAL_RESPONSE_PROMPT.format(research_topic=followup_question_json["query"])
        follow_explanation = query_local_llm(state, config, prompt_initial)
        print("  >> Add new data to notes... \n")
        # Step 5: Add the data to the assay
        state["assay"] =  state["assay"] + follow_explanation

    # print(state["assay"])
    # Step 6: Ask the LLM to finalize the assay by integrating all gathered information and adding references
    prompt_finalize = FINALIZE_RESPONSE_PROMPT.format(research_topic=state["research_topic"], notes=state["assay"] )
    finalize_text_llm = query_local_llm(state, config, prompt_finalize)
    
    print("#### Final Assay ####\n")
    #print(finalize_text_llm)
    display(Markdown(finalize_text_llm))

    # # Step 7: Save the final assay locally
    # try:
    #     with open("final_assay.txt", "w", encoding="utf-8") as f:
    #         f.write(final_assay)
    #     print("\nFinal assay saved to 'final_assay.txt'.")
    # except Exception as e:
    #     print("Error saving the final assay:", e)

if __name__ == "__main__":
    main()

# What is the state of the art of open source LLMs?

#### Research Question ####

how to use LLMs in bioinformatics


